### Resumo livro - Approaching (almost) any Machine Learning Problem


# Métricas de Avaliação

Quando vamos desenvolver algoritmos de machine learning, podemos utilizar diversas métricas de avaliação. Algumas vezes, criamos métricas novas para atender uma necessidade de compreensão para o negócio. Nesse notebook, vamos avaliar as métricas mais comums que podem ser usadas nos mais variados tipos de problemas.

Importante notar que trataremos apensa de métricas de avaliaçao para algotirmos supervisionados devido a grande abundancia desse tipo de problema na indústria e o fato das métricas de avaliação de problemas não supervisionados são um pouco subjetivas.

### As métricas mais usadas para problemas de Classificação são:
* Accuracy 
* Precision (P)
* Recall (R)
* F1 score (F1)
* Area under the ROC (Roceiver Operating Characteristic) curve ou AUC 
* Log Loss
* Precision at k (P@k)
* Average Precision at k (AP@k)
* Mean Average Precision at k (MAP@k)

### As métricas mais usadas para problemas de Regressão são:
* Mean Absolute Error (MAE) 
* Mean Squared Error (MSE)
* Root Mean Squared Error (RMSE)
* Root Mean Squared Logarithmic error (RMSLE)
* Mean Percentage Error (MPE) 
* Mean Absolute Percentage Error (MAPE)
* R²


Saber trabalhar com as métricas mensionadas acima não a única coisa que nós devemos saber. Além disso, devemos saber quando usar quando usar cada uma delas, e isso depende do tipo de dados que temos e do target em questão. 

*Nota: O autor mensiona que se "preocupa" mais com os targets e menos com os tipos de data para saber qual métrica escolher*

Para entender um pouco sobre as métricas, vamos começar com um problema simples de classificação. Vamos supoer que temos um problema de classificação binária ( apenas dois targets) e tal problema consiste em classificar imagens de raio-X toráxica. Temos imagens de raio-x sem problema algum e outras com pneumatorax. Nossa tarefa é montar um algoritmo de classificação que dada uma imagem de raio-x, consiga detectar a presença de pneumatorax. 

.

<img src = 'imagens/pneumotorax.jpg'/>

...

Vamos assumir que temos dados balanceados, ou seja, o número de casos com pneumatorax é igual ao número de casos sem o problema. Se tivermos 100 casos positivos, então teremos 100 casos negativos também.
 

Primeira coisa a se fazer é separar os dados em dois sets iguais de 100 imagens ( set de treino e de validação). Em cada um dos sets, teremos 50 casos positivos e 50 casos negativos para garantir o balanceamento das classes.

> ### Quando temos o mesmo número de classes positivas e negativas num problema de classificação binário,nós usamos as seguintes métricas:
> * Accuracy
> * Precision
> * Recall
> * F1


## Accuracy ou Acurácia

**Accuracy** ou acurácia é a métrica mais simples usada em machine learning. É definido como o quão preciso o modelo é em geral. Para o problema anterior, se a gente constroi um modelo que classifica **90 imagens corretamente**, nossa **accuracy (ou acurácia) é de 90% ou 0.9**. Se somente 83 imagens são classificadas corretamente, a accuracy é de 83% ou 0.83.

### Vamos calcular accuracy com Python:

In [1]:
def accuracy(y_true, y_pred):
    # Inicializa o contador para previsões corretas
    correct = 0
    
    # loop sobre todos elementos de y_true e y_pred 
    for yt, yp in zip(y_true, y_pred):
        if yt == yp:
            correct += 1
    
    # retorna o avalor da accuracy que é a quantidade de previsões corretas sobre o total
    return (correct/ len(y_true))


In [2]:
y_true = [0,1,1,1,0,0,0,1]
y_pred = [0,1,0,1,0,1,0,0]

accuracy_ = accuracy(y_true, y_pred)
print(f"Accuracy : {accuracy_}")

Accuracy : 0.625


* Povemos também realizar o cálculo da acurácia com o scikit-learn:

In [3]:
from sklearn import metrics
y_true = [0,1,1,1,0,0,0,1]
y_pred = [0,1,0,1,0,1,0,0]

accuracy_sklearn = metrics.accuracy_score(y_true,y_pred)

print(f"Accuracy : {accuracy_sklearn}")

Accuracy : 0.625


---

### Dataset Desbalanceado

Agora vamos considerar que temos um dataset desbalanceado com 180 imagens de pacientes que não possuem pneumatórax e apenas 20 que possuem.Mesmo para esse caso, nós iremos criar os datasets de treinamento e validação com as mesmas proporções de targets positivos e negativos. Em cada set, teremos 90 imagems sem pneumatórax e 10 imagens com pneumatórax. *Para a situação em questão, se você disser que todas as imagens do set de validação são de pacientes que não possuem pneumatórax, a acurácia seria de 90%!*

É notório que temos classes desbalanceadas com uma delas significativamente maior que a outra.**Nesse caso, não é recomendado usar a acurácia como métrica de avaliação pois ela não é representativa para os dados em questão**. Mesmo que tenhamos uma acurácia grande, o modelo provavelmente terá uma performance pobre quando for aplicado para dados de produção e você terá sérios problemas para explicar para o seu gerente o motivo.

**Em casos como esse, é melhor olhar para outras métricas como por exemplo a precision.**

Antes de falar em precision, vamos entender outros termos importantes. Vamos assumir que imagem com pneumatórax são da classe positiva (1) e imagens sem são da classe negativa (0).

**True Positive (TP) ou Verdadeiro Positivo (VP)**: Dada uma imagem, se o modelo prever que tal imagem possui pneumatórax e o valor de target da imagem mostra que o paciente realmente possui pneumatórax, é considerado true positive ou verdadeiro positivo.

**True Negative (TN) ou Verdadeiro Negativo (VN)**: Dada uma imagem, se o modelo prever que essa imagem não possui pneumatórax e a imagem é de um paciente que realmente não possui pneumatórax, é considerado True Negative.

Resumindo:
* *Se o modelo prever corretamente classe positiva, é* **True Positive**
* *Se o modelo prever corretamente classe negativa, é* **True Negative**

**False Positive (FP) ou Falso Positivo**: Dada uma imagem, se o modelo prever pneumatórax porém a imagem é de um paciente sem pneumatórax, é um False Positive.

**False Negative (FN) ou Falso Negativo**: Dada uma imagem, se o modelo prever que a imagem não é de paciente com pneumatórax, porém o paciente possui pneumatórax, é um False Negative.

Resumindo:
* *Se o modelo prever incorretamente classe positiva, é* **False Positive**
* *Se o modelo prever incorretamente classe negativa, é* **False Negative**


### Vamos olhar a implementação em python dos conceitos apresentados acima

In [4]:
def true_positive(y_true, y_pred):
    true_pos = 0
    for yt,yp in zip(y_true, y_pred):
        if yt == 1 and yp == 1:
            true_pos += 1
    
    return true_pos

In [5]:
def true_negative(y_true,y_pred):
    true_neg = 0
    for yt,yp in zip(y_true,y_pred):
        if yt == 0 and yp ==0:
            true_neg += 1
            
    return true_neg

In [6]:
def false_positive(y_true, y_pred):
    false_pos = 0
    for yt,yp in zip(y_true, y_pred):
        if yt == 0 and yp ==1:
            false_pos += 1
            
    return false_pos

In [7]:
def false_negative(y_true, y_pred):
    false_neg = 0
    for yt, yp in zip(y_true, y_pred):
        if yt == 1 and yp == 0:
            false_neg += 1
            
    return false_neg

*Nota: As funções aplicadas acima funcionam apenas para classificação binária.*

In [8]:
true_Y = [0,1,1,1,0,0,0,1]
y_Pred = [0,1,0,1,0,1,0,0]

In [9]:
true_positive(true_Y, y_Pred)

2

In [10]:
false_positive(true_Y, y_Pred)

1

In [11]:
false_negative(true_Y, y_Pred)

2

In [12]:
true_negative(true_Y, y_Pred)

3

Se formos definir **Accuracy** com os termos apresentados acima, nós teríamos:

\begin{equation*}
Accuracy Score = \frac{ (TP + TN)}{(TP + TN+FP + FN)}\\
\end{equation*}

Agora que conhecemos todos esses conceito, podemos calcular a acurácia usingo TP, TM, FP, e FN em python.

In [13]:
def accuracy_v2(y_true, y_pred):
    tp = true_positive(y_true, y_pred)
    fp = false_positive(y_true, y_pred)
    fn = false_negative(y_true, y_pred)
    tn = true_negative(y_true, y_pred)
    
    accuracy = (tp + tn)/(tp + tn + fp + fn)
    return accuracy    

> Agora vamos comparar os resultados com a versao do *scikit-learn*

In [14]:
y_true = [0,1,1,1,0,0,0,1]
y_pred = [0,1,0,1,0,1,0,0]

accuracy_1 = accuracy(y_true, y_pred)
print('Accuracy Versão 1: ', accuracy_1)

accuracy_2 = accuracy_v2(y_true, y_pred)
print('Accuracy Versão 2: ', accuracy_2)

from sklearn import metrics
accuracy_scikit_learn = metrics.accuracy_score(y_true, y_pred)
print('Accuracy Versão sklearn: ', accuracy_scikit_learn )

Accuracy Versão 1:  0.625
Accuracy Versão 2:  0.625
Accuracy Versão sklearn:  0.625


Ótimos, agora vimos três maneiras de calcular acurácia e estão todas dando o mesmo resultado.

Agora vamos conhecer melhor e avaliar outras métricas.



A primeira será **Precision**.

---

## Precision

\begin{equation*}
Precision = \frac{ TP }{(TP +FP)}\\
\end{equation*}

Vamos supor que construímos um novo modelo em dados que também são desbalanceados. Diante dessas condições, nosso modelo identificou corretamente 80 casos de não pneumatórax num total de 90 e 8 casos de pneumatórax num total de 10. Consequentemente, o modelo identificou corretamente 88 imagens num total de 100. A **acurácia** nesse caso é de 0,88 ou 88%.

Porém do total de 100 imagens, 10 images sem pneumatórax são classificadas incorretamente como casos com pneumatórax. Além disso, 2 casos de pneumatórax foram  classificados incorretamente como sem pneumatórax.

Diante disso, temos:
>  - TP: 8
>  - TN: 80
>  - FP: 10
>  - FN: 2

Nossa **Precision** será:
    
\begin{equation*}
Precision = \frac{ 8 }{(8 +10)}= 0,444\\ 
\end{equation*} 

> **Isso significa que nosso modelo está correto 44% das vezes quando tenta identificar casos de pneumatórax.**

Agora podemos aplicar precision em python:
    

In [15]:
def precision(y_true, y_pred):
    tp = true_positive(y_true, y_pred)
    fp = false_positive(y_true, y_pred)
    precision = tp / (tp + fp)
    
    return precision

In [16]:
y_true = [0,1,1,1,0,0,0,1]
y_pred = [0,1,0,1,0,1,0,0]

precision_ = precision(y_true, y_pred)
print('precision:', precision_)

precision: 0.6666666666666666


Agora podemos analisar **Recall**

---

## Recall

**Recall** é definido como:
    
\begin{equation*}
Recall = \frac{ TP }{(TP + FN)}\\ 
\end{equation*}

Nossa **Recall** será:
    
\begin{equation*}
Precision = \frac{ 8 }{(8 + 2)}= 0,80\\ 
\end{equation*}

> **Isso significa que nosso modelo identificou 80% dos casos positivos de pneumatórax corretamente**

 Vamos aplicar o cálculo em python.

In [17]:
def recall(y_true, y_pred):
    tp = true_positive(y_true, y_pred)
    fn = false_negative(y_true, y_pred)
    
    recall = tp/ (tp + fn)
    
    return recall

In [18]:
y_true = [0,1,1,1,0,0,0,1]
y_pred = [0,1,0,1,0,1,0,0]

recall_ = recall(y_true, y_pred)
print('recall:', recall_)

recall: 0.5


Para um bom modelo de machine learning, os valores de precision e recall precisam ser altos. Podemos observar isso no caso anterior, o valor de recall é 'um pouco' alto. Em contrapartida o valor de precision é muito baixo!

Nosso modelo reproduz bastante falso positivos porém menos faso negativos. Poucos casos de falso negativos é bom nesse tipo de problema pois não queremos dizer que um paciente não possui pneumatórax quando na verdade ele possui. Isso pode proporcionar sérios problemas ! Porém ainda temos bastante falso positivos e isso também pode ser um problema.

grande parte dos modelos prevê probabilidade. Nessas predições nós geralmente escolhemos um threshold de 0,5. Esse limite (threshold) nao é sempre ideal e dependendo do valor escolhido para esse limite, os valores de precision e recall podem mudar drasticamente. Se para cada threshold que a gente escolher, calcularmos os valores de precision e recall, nos criamos uma

## Curva de Precision e Recall

Vamos assumir que temos duas listas:
* y_true: os valores de targets
* y_pred: Valores de probabilidade da amostra em questão ser da classe 1

Agora vamos olhar as probabilidades de predições ao invés do valor que foi predito ( o que é calculado na grande maioria das vezes considerando um limite de 0.5)

In [25]:
y_true = [0,0,0,1,0,0,0,0,0,0,
          1,0,0,0,0,0,0,0,1,0]


y_pred = [0.263 , 0.111, 0.032, 
          0.049 , 0.019, 0.175,
          0.159 , 0.038, 0.116,
          0.079 , 0.085, 0.391,
          0.272 , 0.034, 0.046,
          0.035 , 0.185, 0.059,
          0.612 , 0.033]

In [26]:
print(len(y_true))

print(len(y_pred))

20
20
